In [1]:
!git clone https://github.com/LynnHaDo/Checkbox-Detection.git

fatal: destination path 'Checkbox-Detection' already exists and is not an empty directory.


In [1]:
%pip install opencv-python
%pip install matplotlib
%pip install numpy
%pip install albumentations
%pip install ultralytics==8.0.230
%pip install gradio
%pip install torch
%pip install ruamel
%pip install easyocr

  Using cached opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata (20 kB)
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------------------------- ------------ 41.0/61.0 kB 991.0 kB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 819.4 kB/s eta 0:00:00
Using cached opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl (38.6 MB)
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/15.8 MB 9.8 MB/s eta 0:00:02
   -- ------------------------------------- 1.0/15.8 MB 10.5 MB/s eta 0:00:02
   --- ------------------------------------ 1.5/15.8 MB 10.7 MB/s eta 0:00:02
   ----- ---------------------------------- 2.1/15.8 MB 11.0 MB/s eta 0:00:02
   ------ --------------------------------- 2.6/15.8 MB 11.0 MB/s eta 0:00:02
   ------- -------------------------------- 3.1/15.8 MB 11.1 MB/s eta 0:00:02
   --------- ------------------------------ 3.7/15.8 MB 11.2 MB/s eta 0:00:02
   -

ERROR: Could not find a version that satisfies the requirement ruamel (from versions: none)
ERROR: No matching distribution found for ruamel


In [201]:
import cv2 # for reading images, draw bounding boxes
from ultralytics import YOLO 
import gradio as gr
import pandas as pd
import easyocr
import numpy as np

In [3]:
# Define constants
BOX_COLORS = {
    "unchecked": (242, 48, 48),
    "checked": (38, 115, 101),
    "block": (242, 159, 5)
}
BOX_PADDING = 2


In [4]:
import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

In [5]:
# Load models
DETECTION_MODEL = YOLO("../models/detector-model.pt") 
CLASSIFICATION_MODEL = YOLO("../models/classifier-model.pt")

WARNING ⚠️ ../models/classifier-model.pt appears to require 'dill', which is not in ultralytics requirements.
AutoInstall will run now for 'dill' but this feature will be removed in the future.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official YOLOv8 model, i.e. 'yolo predict model=yolov8n.pt'
requirements: Ultralytics requirement ['dill'] not found, attempting AutoUpdate...
   ---------------------------------------- 116.3/116.3 kB 2.3 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 4.1s, installed 1 package: ['dill']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



# With Gradio UI

In [ ]:
def detect(image_path):
    """
    Output inference image with bounding box
    Args:
    - image: to check for checkboxes
    Return: image with bounding boxes drawn
    """
    image = cv2.imread(image_path)
    if image is None:
        return image
    
    # Predict on image
    results = DETECTION_MODEL.predict(source=image, conf=0.01, iou=0.8) # Predict on image
    boxes = results[0].boxes # Get bounding boxes

    if len(boxes) == 0:
        return image

    data = []
    reader = easyocr.Reader(['en']) # this needs to run only once to load the model into memory
    result = reader.readtext(image_path)

    for i,box in enumerate(boxes):
        detection_class_conf = round(box.conf.item(), 2)
        detection_class = list(BOX_COLORS)[int(box.cls)]
        # Get start and end points of the current box
        start_box = (int(box.xyxy[0][0]), int(box.xyxy[0][1]))
        end_box = (int(box.xyxy[0][2]), int(box.xyxy[0][3]))
        ibox = image[start_box[1]:end_box[1], start_box[0]: end_box[0], :]

        # Determine the class of the box using classification model
        cls_results = CLASSIFICATION_MODEL.predict(source=ibox, conf=0.5)
        probs = cls_results[0].probs  # cls prob, (num_class, )
        classification_class = list(BOX_COLORS)[2 - int(probs.top1)] 
        classification_class_conf = round(probs.top1conf.item(), 2)

        cls = classification_class if classification_class_conf > 0.9 else detection_class
        
        #centerpoint
        bbox = box.xyxy
        x = ((bbox[:,0] + bbox[:,2]) / 2)
        y = (bbox[:,1] + bbox[:,3]) / 2 
        x,y = [int(x),int(y)]


        
        if cls == 'checked':
            left_text = []
            left_text_bboxes = [] 
            # To store bounding boxes of left text
            for detection in result:
                bbox = detection[0]  # Get bounding box of text region (format: [[x1, y1], [x2, y2], [x3, y3], [x4, y4]])
                centroid_x = sum([p[0] for p in bbox]) / 4  # Calculate centroid x-coordinate
                centroid_y = sum([p[1] for p in bbox]) / 4  # Calculate centroid y-coordinate
                if centroid_x < x:  # Check if text is on the left side of (x, y)
                    left_text.append(detection[1])  # Append text to list
                    left_text_bboxes.append(bbox)  # Append bounding box to list

            # Find the bounding box closest to the point (x, y)
            closest_bbox = None
            closest_distance = float('inf')
            index = 0
            for i,bbox in enumerate(left_text_bboxes):
                centroid_x = sum([p[0] for p in bbox]) / 4
                centroid_y = sum([p[1] for p in bbox]) / 4
                distance = np.sqrt((centroid_x - x)**2 + (centroid_y - y)**2)
                if distance < closest_distance and centroid_y<y+15 and centroid_y>y-15 :
                    closest_bbox = bbox
                    closest_distance = distance
                    index = i
            # Draw bounding box on the image
            if closest_bbox is not None:
                bbox = [[int(p[0]), int(p[1])] for p in closest_bbox]  # Convert float coordinates to int
                for i in range(4):
                    cv2.line(image, tuple(bbox[i]), tuple(bbox[(i + 1) % 4]), (255, 255, 0), 2)
            cv2.circle(image, (x, y), 5, (0, 255, 0), -1)

            
            data.append({'class': cls, 'checkbox_coords': [x, y], 'closest_text': left_text[index]})
        
    
    
    
    return pd.DataFrame(data)

iface = gr.Interface(fn=detect, 
                     inputs=gr.Image(label="Upload scanned document", type="filepath"), 
                     outputs=gr.Dataframe(label="image"))
iface.launch(share=True)


Running on local URL:  http://127.0.0.1:7865
Running on public URL: https://c8590291df0861aafe.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)



0: 640x480 20 uncheckeds, 7 checkeds, 6 msts, 395.9ms
Speed: 13.0ms preprocess, 395.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.



0: 32x32 block 1.00, unchecked 0.00, checked 0.00, 6.6ms
Speed: 1.0ms preprocess, 6.6ms inference, 0.0ms postprocess per image at shape (1, 3, 32, 32)

0: 32x32 checked 1.00, block 0.00, unchecked 0.00, 5.5ms
Speed: 0.0ms preprocess, 5.5ms inference, 0.0ms postprocess per image at shape (1, 3, 32, 32)

0: 32x32 block 1.00, unchecked 0.00, checked 0.00, 5.0ms
Speed: 0.0ms preprocess, 5.0ms inference, 0.0ms postprocess per image at shape (1, 3, 32, 32)

0: 32x32 unchecked 1.00, checked 0.00, block 0.00, 5.0ms
Speed: 1.5ms preprocess, 5.0ms inference, 0.0ms postprocess per image at shape (1, 3, 32, 32)

0: 32x32 block 1.00, checked 0.00, unchecked 0.00, 5.5ms
Speed: 0.0ms preprocess, 5.5ms inference, 0.0ms postprocess per image at shape (1, 3, 32, 32)

0: 32x32 unchecked 0.99, block 0.01, checked 0.00, 5.0ms
Speed: 1.5ms preprocess, 5.0ms inference, 0.0ms postprocess per image at shape (1, 3, 32, 32)

0: 32x32 block 0.99, checked 0.00, unchecked 0.00, 6.3ms
Speed: 1.0ms preprocess, 6.3ms

In [218]:
image = cv2.imread('../src/images/image.jpg')


# Predict on image
results = DETECTION_MODEL.predict(source=image, conf=0.01, iou=0.8) # Predict on image
boxes = results[0].boxes # Get bounding boxes




data = []

reader = easyocr.Reader(['en']) # this needs to run only once to load the model into memory
result = reader.readtext('../src/images/image.jpg')


# Get bounding boxes
for i,box in enumerate(boxes):
    detection_class_conf = round(box.conf.item(), 2)
    detection_class = list(BOX_COLORS)[int(box.cls)]
    # Get start and end points of the current box
    start_box = (int(box.xyxy[0][0]), int(box.xyxy[0][1]))
    end_box = (int(box.xyxy[0][2]), int(box.xyxy[0][3]))
    ibox = image[start_box[1]:end_box[1], start_box[0]: end_box[0], :]

    # Determine the class of the box using classification model
    cls_results = CLASSIFICATION_MODEL.predict(source=ibox, conf=0.5)
    probs = cls_results[0].probs  # cls prob, (num_class, )
    classification_class = list(BOX_COLORS)[2 - int(probs.top1)] 
    classification_class_conf = round(probs.top1conf.item(), 2)

    cls = classification_class if classification_class_conf > 0.9 else detection_class
    
    #centerpoint
    bbox = box.xyxy
    x = ((bbox[:,0] + bbox[:,2]) / 2)
    y = (bbox[:,1] + bbox[:,3]) / 2 
    x,y = [int(x),int(y)]


    
    if cls == 'checked':
        left_text = []
        left_text_bboxes = [] 
        # To store bounding boxes of left text
        for detection in result:
            bbox = detection[0]  # Get bounding box of text region (format: [[x1, y1], [x2, y2], [x3, y3], [x4, y4]])
            centroid_x = sum([p[0] for p in bbox]) / 4  # Calculate centroid x-coordinate
            centroid_y = sum([p[1] for p in bbox]) / 4  # Calculate centroid y-coordinate
            if centroid_x < x:  # Check if text is on the left side of (x, y)
                left_text.append(detection[1])  # Append text to list
                left_text_bboxes.append(bbox)  # Append bounding box to list

        # Find the bounding box closest to the point (x, y)
        closest_bbox = None
        closest_distance = float('inf')
        index = 0
        for i,bbox in enumerate(left_text_bboxes):
            centroid_x = sum([p[0] for p in bbox]) / 4
            centroid_y = sum([p[1] for p in bbox]) / 4
            distance = np.sqrt((centroid_x - x)**2 + (centroid_y - y)**2)
            if distance < closest_distance and centroid_y<y+15 and centroid_y>y-15 :
                closest_bbox = bbox
                closest_distance = distance
                index = i
        # Draw bounding box on the image
        if closest_bbox is not None:
            bbox = [[int(p[0]), int(p[1])] for p in closest_bbox]  # Convert float coordinates to int
            for i in range(4):
                cv2.line(image, tuple(bbox[i]), tuple(bbox[(i + 1) % 4]), (255, 255, 0), 2)
        cv2.circle(image, (x, y), 5, (0, 255, 0), -1)

        
        data.append({'class': cls, 'checkbox_coords': [x, y], 'closest_text': left_text[index]})
    
cv2.imwrite('test.jpg',image)



0: 640x480 19 uncheckeds, 8 checkeds, 6 msts, 381.7ms
Speed: 3.0ms preprocess, 381.7ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.



0: 32x32 checked 1.00, unchecked 0.00, block 0.00, 6.0ms
Speed: 1.0ms preprocess, 6.0ms inference, 0.0ms postprocess per image at shape (1, 3, 32, 32)

0: 32x32 unchecked 0.97, block 0.03, checked 0.00, 5.5ms
Speed: 1.0ms preprocess, 5.5ms inference, 0.0ms postprocess per image at shape (1, 3, 32, 32)

0: 32x32 block 1.00, unchecked 0.00, checked 0.00, 5.0ms
Speed: 0.0ms preprocess, 5.0ms inference, 0.0ms postprocess per image at shape (1, 3, 32, 32)

0: 32x32 block 0.98, unchecked 0.01, checked 0.01, 5.5ms
Speed: 1.0ms preprocess, 5.5ms inference, 0.0ms postprocess per image at shape (1, 3, 32, 32)

0: 32x32 checked 0.95, block 0.05, unchecked 0.00, 6.5ms
Speed: 1.0ms preprocess, 6.5ms inference, 0.0ms postprocess per image at shape (1, 3, 32, 32)

0: 32x32 block 1.00, checked 0.00, unchecked 0.00, 5.1ms
Speed: 1.0ms preprocess, 5.1ms inference, 0.0ms postprocess per image at shape (1, 3, 32, 32)

0: 32x32 block 1.00, unchecked 0.00, checked 0.00, 5.1ms
Speed: 1.0ms preprocess, 5.1ms

True

In [219]:
data

[{'class': 'checked',
  'checkbox_coords': [274, 935],
  'closest_text': 'Cilizenship: Maleysia'},
 {'class': 'checked',
  'checkbox_coords': [694, 1060],
  'closest_text': 'indian'},
 {'class': 'checked',
  'checkbox_coords': [506, 1035],
  'closest_text': 'Female'},
 {'class': 'checked', 'checkbox_coords': [263, 1060], 'closest_text': 'Hindu'},
 {'class': 'checked',
  'checkbox_coords': [747, 1590],
  'closest_text': 'Auto Debilf BSM MAuto Debit MBB'},
 {'class': 'checked',
  'checkbox_coords': [834, 999],
  'closest_text': 'No. EJeN'},
 {'class': 'checked',
  'checkbox_coords': [692, 1061],
  'closest_text': 'indian'},
 {'class': 'checked',
  'checkbox_coords': [506, 1028],
  'closest_text': 'Female'}]

In [197]:
import easyocr
import numpy as np
reader = easyocr.Reader(['en']) # this needs to run only once to load the model into memory
result = reader.readtext('test.jpg')


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [208]:
vertical_tolerance = 50
x, y=694, 1060

# Without yapping

In [209]:
image = cv2.imread('../src/images/image.jpg')
# Filter text on the left side of (x, y)
left_text = []
left_text_bboxes = []  # To store bounding boxes of left text
for detection in result:
    bbox = detection[0]  # Get bounding box of text region (format: [[x1, y1], [x2, y2], [x3, y3], [x4, y4]])
    centroid_x = sum([p[0] for p in bbox]) / 4  # Calculate centroid x-coordinate
    centroid_y = sum([p[1] for p in bbox]) / 4  # Calculate centroid y-coordinate
    if centroid_x < x:  # Check if text is on the left side of (x, y)
        left_text.append(detection[1])  # Append text to list
        left_text_bboxes.append(bbox)  # Append bounding box to list

# Find the bounding box closest to the point (x, y)
closest_bbox = None
closest_distance = float('inf')
for bbox in left_text_bboxes:
    centroid_x = sum([p[0] for p in bbox]) / 4
    centroid_y = sum([p[1] for p in bbox]) / 4
    distance = np.sqrt((centroid_x - x)**2 + (centroid_y - y)**2)
    if distance < closest_distance:
        closest_bbox = bbox
        closest_distance = distance
# Draw bounding box on the image
if closest_bbox is not None:
    bbox = [[int(p[0]), int(p[1])] for p in closest_bbox]  # Convert float coordinates to int
    for i in range(4):
        cv2.line(image, tuple(bbox[i]), tuple(bbox[(i + 1) % 4]), (255, 255, 0), 2)

In [210]:
cv2.imwrite('text.jpg',image)

True